In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

In [2]:
from pandasql import sqldf

In [3]:
from pandasql import sqldf, load_meat, load_births

In [4]:
pysqldf = lambda q: sqldf(q, globals())

In [5]:
meat = load_meat()

In [6]:
meat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827 entries, 0 to 826
Data columns (total 8 columns):
date               827 non-null datetime64[ns]
beef               827 non-null float64
veal               827 non-null float64
pork               827 non-null float64
lamb_and_mutton    827 non-null float64
broilers           635 non-null float64
other_chicken      143 non-null float64
turkey             635 non-null float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 51.8 KB


In [7]:
meat.head()

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01,751.0,85.0,1280.0,89.0,NaN,NaN,NaN
1,1944-02-01,713.0,77.0,1169.0,72.0,NaN,NaN,NaN
2,1944-03-01,741.0,90.0,1128.0,75.0,NaN,NaN,NaN
3,1944-04-01,650.0,89.0,978.0,66.0,NaN,NaN,NaN
4,1944-05-01,681.0,106.0,1029.0,78.0,NaN,NaN,NaN


In [8]:
births = load_births()

In [9]:
births.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 2 columns):
date      408 non-null datetime64[ns]
births    408 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.5 KB


In [10]:
births.head()

,date,births
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545


In [11]:
pysqldf("SELECT * FROM meat LIMIT 5;")

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01 00:00:00.000000,751.0,85.0,1280.0,89.0,None,None,None
1,1944-02-01 00:00:00.000000,713.0,77.0,1169.0,72.0,None,None,None
2,1944-03-01 00:00:00.000000,741.0,90.0,1128.0,75.0,None,None,None
3,1944-04-01 00:00:00.000000,650.0,89.0,978.0,66.0,None,None,None
4,1944-05-01 00:00:00.000000,681.0,106.0,1029.0,78.0,None,None,None


In [12]:
pysqldf("SELECT * FROM births LIMIT 5;")

,date,births
0,1975-01-01 00:00:00.000000,265775
1,1975-02-01 00:00:00.000000,241045
2,1975-03-01 00:00:00.000000,268849
3,1975-04-01 00:00:00.000000,247455
4,1975-05-01 00:00:00.000000,254545


In [13]:
#let's work on the joins and aggregations

In [14]:
# execution of simple group by query
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;"""

In [15]:
print(q)

SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;


In [16]:
pysqldf(q)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0


In [17]:
# you want to print all the results and then do selection of top 10 records
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year;"""

In [18]:
pysqldf(q).head(10)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0
5,1949,9142.0
6,1950,9248.0
7,1951,8549.0
8,1952,9337.0
9,1953,12055.0


In [19]:
# order by using random functions
q = """SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;"""


In [20]:
print(q)

SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;


In [21]:
print (pysqldf(q))

                         date    beef   veal    pork  lamb_and_mutton  \
0  2000-06-01 00:00:00.000000  2369.0   18.0  1538.0             17.0   
1  1952-10-01 00:00:00.000000   933.0  128.0   894.0             68.0   
2  1972-12-01 00:00:00.000000  1813.0   31.0  1087.0             42.0   
3  1972-04-01 00:00:00.000000  1717.0   34.0  1139.0             44.0   
4  2011-05-01 00:00:00.000000  2131.9   10.3  1759.7             12.9   
5  1950-08-01 00:00:00.000000   829.0  108.0   633.0             52.0   
6  1996-09-01 00:00:00.000000  1934.0   31.0  1408.0             20.0   
7  1972-07-01 00:00:00.000000  1692.0   33.0   902.0             37.0   
8  1986-04-01 00:00:00.000000  2110.0   45.0  1292.0             29.0   
9  1978-02-01 00:00:00.000000  1954.0   56.0  1013.0             23.0   

   broilers  other_chicken  turkey  
0    2672.2            NaN   483.4  
1       NaN            NaN     NaN  
2     582.2            NaN   152.7  
3     623.7            NaN    44.3  
4    3251.1

In [22]:
#executing union all statements
q = """
        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    """

In [23]:
print(q)


        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    


In [24]:
print (pysqldf(q).head(10))

                         date        meat_type   value
0  1944-01-01 00:00:00.000000             beef   751.0
1  1944-01-01 00:00:00.000000             veal    85.0
2  1944-01-01 00:00:00.000000             pork  1280.0
3  1944-01-01 00:00:00.000000  lamb_and_mutton    89.0
4  1944-02-01 00:00:00.000000             beef   713.0
5  1944-02-01 00:00:00.000000             veal    77.0
6  1944-02-01 00:00:00.000000             pork  1169.0
7  1944-02-01 00:00:00.000000  lamb_and_mutton    72.0
8  1944-03-01 00:00:00.000000             beef   741.0
9  1944-03-01 00:00:00.000000             veal    90.0


In [25]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;
"""

In [26]:
print(q)


    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;



In [27]:
print(pysqldf(q).head())

                         date    beef  veal    pork  lamb_and_mutton  \
0  1975-01-01 00:00:00.000000  2106.0  59.0  1114.0             36.0   
1  1975-02-01 00:00:00.000000  1845.0  50.0   954.0             31.0   
2  1975-03-01 00:00:00.000000  1891.0  57.0   976.0             35.0   
3  1975-04-01 00:00:00.000000  1895.0  60.0  1100.0             34.0   
4  1975-05-01 00:00:00.000000  1849.0  59.0   934.0             31.0   

   broilers  other_chicken  turkey  births  
0     646.2            NaN    64.9  265775  
1     570.2            NaN    47.1  241045  
2     616.6            NaN    54.4  268849  
3     688.3            NaN    68.7  247455  
4     690.1            NaN    81.9  254545  


In [28]:
#Perform inner join on two tables
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;"""

In [29]:
print(q)

SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;


In [30]:
#after join, print first 10 records
pysqldf(q).head(10)

,date,beef,births
0,1975-01-01 00:00:00.000000,2106.0,265775
1,1975-02-01 00:00:00.000000,1845.0,241045
2,1975-03-01 00:00:00.000000,1891.0,268849
3,1975-04-01 00:00:00.000000,1895.0,247455
4,1975-05-01 00:00:00.000000,1849.0,254545
5,1975-06-01 00:00:00.000000,1849.0,254096
6,1975-07-01 00:00:00.000000,1916.0,275163
7,1975-08-01 00:00:00.000000,1961.0,281300
8,1975-09-01 00:00:00.000000,2065.0,270738
9,1975-10-01 00:00:00.000000,2270.0,265494


In [31]:
# use queries within queries
q = """
    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)
"""

In [32]:
print(q)


    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)



In [33]:
pysqldf(q).head()

,date,beef
0,1960-01-01 00:00:00.000000,1196.0
1,1960-02-01 00:00:00.000000,1089.0
2,1960-03-01 00:00:00.000000,1201.0
3,1960-04-01 00:00:00.000000,1066.0
4,1960-05-01 00:00:00.000000,1202.0
